In [1]:

import matplotlib_venn
import pandas as pd
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
from Bio import SeqIO
from collections import defaultdict

def read_transdecoder_nucleotides_file(transdecoder_nucleotides_file):
    transdecorder_sequences = {}
    for record in SeqIO.parse(transdecoder_nucleotides_file, 'fasta'):
        acc = record.id.split('.')[:-1]
        acc = '.'.join(acc)
        transdecorder_sequences[acc] = record.seq
    return transdecorder_sequences

def read_cpat_nucleotides_file(cpat_best_file, cpat_nucleotides_file):
    cpat_best = pd.read_table(cpat_best_file)
    best_orfs = set(cpat_best['ID'])
    cpat_sequences = {}
    for record in SeqIO.parse(cpat_nucleotides_file,'fasta'):
        if record.id in best_orfs:
            accession = record.id.split('_')[0]
            cpat_sequences[accession] = record.seq
    return cpat_sequences

def compare_sequences(cpat, transdecoder):
    comparisons = []
    accessions = set(cpat.keys()).union(set(transdecoder.keys()))
    for accession in accessions:
        accession_comparison = ''
        if accession not in cpat.keys():
            accession_comparison = 'not_found_in_cpat'
        elif accession not in transdecoder.keys():
            accession_comparison = 'not_found_in_transdecoder'
        elif cpat[accession] == transdecoder[accession]:
            accession_comparison = 'same_orf_called'
        else:
            accession_comparison = 'different_orfs_called'
        comparisons.append([accession, accession_comparison])
    comparisons = pd.DataFrame(comparisons, columns=['PacBio Accession','Comparison'])
    return comparisons


In [41]:

transdecoder_nucleotides_file = '/Users/bj8th/Documents/Sheynkman-Lab/GitHub/Long-Read-Proteogenomics-Analysis/revisions/transdecoder/analysis/jurkat_corrected.5degfilter.fasta.transdecoder.cds'
cpat_best_file = '/Users/bj8th/Documents/Sheynkman-Lab/Data/JURKAT_06-06-2021/cpat/jurkat.ORF_prob.best.tsv'
cpat_nucleotides_file = '/Users/bj8th/Documents/Sheynkman-Lab/Data/JURKAT_06-06-2021/cpat/jurkat.ORF_seqs.fa'
sqanti_classifications_file = '/Users/bj8th/Documents/Sheynkman-Lab/Data/JURKAT_06-06-2021/sqanti3-filtered/jurkat_classification.5degfilter.tsv'

transdecorder_sequences = read_transdecoder_nucleotides_file(transdecoder_nucleotides_file)
cpat_sequences = read_cpat_nucleotides_file(cpat_best_file, cpat_nucleotides_file)

comparisions = compare_sequences(cpat_sequences, transdecorder_sequences)


def get_all_cpat_orfs(cpat_nucleotides_file):
    cpat_sequences = defaultdict(list)
    for record in SeqIO.parse(cpat_nucleotides_file,'fasta'):
        accession = record.id.split('_')[0]
        cpat_sequences[accession].append(str(record.seq))
    return cpat_sequences

def transdecoder_orf_found_in_cpat(accession, transdecoder, cpat_full):
    transdecoder_seq = str(transdecoder[accession])

    return transdecoder_seq in cpat_full[accession]
comparisions['cpat_start'] = comparisions['PacBio Accession'].apply(lambda acc: str(cpat_sequences[acc][:3]) if acc in cpat_sequences.keys() else 'N/A')
comparisions['transdecoder_start'] = comparisions['PacBio Accession'].apply(lambda acc: str(transdecorder_sequences[acc][:3]) if acc in transdecorder_sequences.keys() else 'N/A')
classification = pd.read_table(sqanti_classifications_file)
comparisions = comparisions.merge(classification[['isoform', 'structural_category']], left_on='PacBio Accession', right_on='isoform')
cpat_full = get_all_cpat_orfs(cpat_nucleotides_file)

different_orfs = comparisions[comparisions['Comparison'] =='different_orfs_called']
different_orfs['transdecoder_sequence_found_in_cpat'] = different_orfs['PacBio Accession'].apply(lambda acc: transdecoder_orf_found_in_cpat(acc, transdecorder_sequences, cpat_full))


<ipython-input-41-833ddc4f6162>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  different_orfs['transdecoder_sequence_found_in_cpat'] = different_orfs['PacBio Accession'].apply(lambda acc: transdecoder_orf_found_in_cpat(acc, transdecorder_sequences, cpat_full))


In [42]:
not_found_in_cpat = different_orfs[~different_orfs['transdecoder_sequence_found_in_cpat']]

In [43]:
not_found_in_cpat.groupby('structural_category').size()

structural_category
full-splice_match          11207
incomplete-splice_match     6643
novel_in_catalog           10893
novel_not_in_catalog        8150
dtype: int64

In [23]:
not_found_in_cpat['transdecoder_start'] = not_found_in_cpat['PacBio Accession'].apply(lambda acc: str(transdecorder_sequences[acc][:3]) if acc in transdecorder_sequences.keys() else 'N/A')
not_found_in_cpat['transdecoder_ATG'] = not_found_in_cpat['start'].apply(lambda start: start =='ATG')
not_found_in_cpat.groupby('transdecoder_ATG').size().sort_values(ascending=False) / len(not_found_in_cpat)

<ipython-input-23-7d04d8183d40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_found_in_cpat['transdecoder_start'] = not_found_in_cpat['PacBio Accession'].apply(lambda acc: str(transdecorder_sequences[acc][:3]))
<ipython-input-23-7d04d8183d40>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_found_in_cpat['transdecoder_ATG'] = not_found_in_cpat['start'].apply(lambda start: start =='ATG')


transdecoder_ATG
False    0.725151
True     0.274849
dtype: float64

In [24]:
not_found_in_cpat.groupby('transdecoder_start').size().sort_values(ascending=False)

transdecoder_start
ATG    10140
GGC     1212
GAG     1209
GGG     1143
GCG     1087
       ...  
ATA       90
CTA       83
TTA       83
TAC       75
TAT       52
Length: 61, dtype: int64

In [22]:


comparisions.groupby('cpat_start').size()

cpat_start
ATG    139734
N/A         8
dtype: int64

In [35]:
comparisions.groupby(['Comparison', 'structural_category']).size()

Comparison                 structural_category    
different_orfs_called      full-splice_match          11985
                           incomplete-splice_match     7317
                           novel_in_catalog           13158
                           novel_not_in_catalog        9379
not_found_in_cpat          full-splice_match              3
                           incomplete-splice_match        1
                           novel_in_catalog               3
                           novel_not_in_catalog           1
not_found_in_transdecoder  full-splice_match            181
                           incomplete-splice_match      150
                           novel_in_catalog             363
                           novel_not_in_catalog         135
same_orf_called            full-splice_match          31696
                           incomplete-splice_match    12918
                           novel_in_catalog           29551
                           novel_not_in_catalog  

In [47]:
cpat_atg = comparisions['cpat_start'] =='ATG'
tcoder_atg = comparisions['transdecoder_start'] =='ATG'
atg_start = comparisions[cpat_atg & tcoder_atg]
atg_start.groupby(['Comparison',]).size() / len(atg_start)

Comparison
different_orfs_called    0.134514
same_orf_called          0.865486
dtype: float64

In [49]:
atg_diff = atg_start[atg_start['Comparison'] =='different_orfs_called']
atg_diff['transdecoder_sequence_found_in_cpat'] = atg_diff['PacBio Accession'].apply(lambda acc: transdecoder_orf_found_in_cpat(acc, transdecorder_sequences, cpat_full))
atg_diff.groupby('transdecoder_sequence_found_in_cpat').size()

<ipython-input-49-c388b0b02761>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atg_diff['transdecoder_sequence_found_in_cpat'] = atg_diff['PacBio Accession'].apply(lambda acc: transdecoder_orf_found_in_cpat(acc, transdecorder_sequences, cpat_full))


transdecoder_sequence_found_in_cpat
False    10140
True      4946
dtype: int64

,PacBio Accession,Comparison,cpat_start,transdecoder_start,isoform,structural_category,transdecoder_sequence_found_in_cpat
11,PB.13770.3,different_orfs_called,ATG,ATG,PB.13770.3,full-splice_match,False
12,PB.12581.48,different_orfs_called,ATG,ATG,PB.12581.48,novel_in_catalog,False
17,PB.15151.4,different_orfs_called,ATG,ATG,PB.15151.4,novel_not_in_catalog,True
21,PB.9118.14,different_orfs_called,ATG,ATG,PB.9118.14,novel_in_catalog,True
24,PB.5982.36,different_orfs_called,ATG,ATG,PB.5982.36,novel_in_catalog,False
...,...,...,...,...,...,...,...
139729,PB.14427.1,different_orfs_called,ATG,ATG,PB.14427.1,full-splice_match,True
139730,PB.14276.17,different_orfs_called,ATG,ATG,PB.14276.17,novel_not_in_catalog,False
139733,PB.12350.10,different_orfs_called,ATG,ATG,PB.12350.10,novel_in_catalog,False
139735,PB.15552.6,different_orfs_called,ATG,ATG,PB.15552.6,novel_in_catalog,False


In [53]:
len(cpat_sequences['PB.13770.3'])

285

In [54]:
len(transdecorder_sequences['PB.13770.3'])

678

In [58]:
for seq in cpat_full['PB.13770.3']:
    if seq in transdecorder_sequences['PB.13770.3']:
        print('subset')

subset
subset
subset
subset
subset
subset
subset
subset
subset
subset
subset
subset
subset
subset


In [60]:
import os
utilitiesPath = '/Users/bj8th/Documents/Sheynkman-Lab/GitHub/SQANTI3/utilities'
GMSP_PROG = os.path.join(utilitiesPath, "gmst", "gmst.pl")
GMST_CMD = "perl " + GMSP_PROG + " -faa --strand direct --fnn --output {o} {i}"
input_fasta = '/Users/bj8th/Documents/Sheynkman-Lab/Data/JURKAT_06-06-2021/sqanti3-filtered/jurkat_corrected.5degfilter.fasta'
output_dir = '/Users/bj8th/Documents/Sheynkman-Lab/GitHub/Long-Read-Proteogenomics-Analysis/revisions/gmst/results'
cmd = GMST_CMD.format(i=os.path.realpath(input_fasta), o=output_dir)
cmd

'perl /Users/bj8th/Documents/Sheynkman-Lab/GitHub/SQANTI3/utilities/gmst/gmst.pl -faa --strand direct --fnn --output /Users/bj8th/Documents/Sheynkman-Lab/GitHub/Long-Read-Proteogenomics-Analysis/revisions/gmst/results /Users/bj8th/Documents/Sheynkman-Lab/Data/JURKAT_06-06-2021/sqanti3-filtered/jurkat_corrected.5degfilter.fasta'